<a href="https://colab.research.google.com/github/atikhasan007/Scikit-learn/blob/main/kaggol%20copy%20code%20and%20help%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import os
import torch
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import time

def create_mask_from_npz(npz_file_path):

    """Creates a binary mask from polygons stored in an NPZ file.

    Args:
        npz_file_path (str): Path to the NPZ file containing polygon data.

    Returns:
        np.ndarray: The binary mask (256x256).
    """

    # Load data from the NPZ file
    with np.load(npz_file_path) as f:
        # Get all arrays from the NPZ file
        polygon_arrays = [v for k, v in f.items() if k.startswith('arr_')]

    # Create an empty mask (black) with the fixed dimensions
    mask = np.zeros((256, 256), dtype=np.uint8)  # Use uint8 for binary mask

    # Iterate over each polygon array and fill the mask
    for polygons in polygon_arrays:
        if polygons.ndim == 2 and polygons.shape[1] == 2:  # Check if valid polygon format
            mask = cv2.fillPoly(mask, pts=[polygons], color=255) # White for filled area

    # Apply Gaussian blur for anti-aliasing (optional)
    #mask = cv2.GaussianBlur(mask, (0, 0), sigmaX=2, sigmaY=2, borderType=cv2.BORDER_DEFAULT)
    mask = mask / mask.max()
    return mask

# usage
npz_file = "/kaggle/input/preprocessed-change/change_detection_dataset_poly_preprocessed/label/79114_50991.npz"
result_mask = create_mask_from_npz(npz_file)

# can save or display the results
plt.imshow(result_mask,cmap="gray")


In [ ]:
def __init__(self, image_dir_A, image_dir_B, mask_dir, transform=None):
        self.image_dir_A = image_dir_A
        self.image_dir_B = image_dir_B
        sclass ChangeDetectionDataset(Dataset):
 elf.mask_dir = mask_dir
        self.transform = transform
        self.image_names = os.listdir(image_dir_A)

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_A_name = self.image_names[idx]
        img_B_name = img_A_name  # Assuming both image sets have the same filenames

        img_A_path = os.path.join(self.image_dir_A, img_A_name)
        img_B_path = os.path.join(self.image_dir_B, img_B_name)
        mask_path = os.path.join(self.mask_dir, os.path.splitext(img_A_name)[0] + '.npz')

        image_A = Image.open(img_A_path).convert('RGB')
        image_B = Image.open(img_B_path).convert('RGB')

        mask = create_mask_from_npz(mask_path)

        if self.transform:
            image_A = self.transform(image_A)
            image_B = self.transform(image_B)
            mask = torch.from_numpy(mask).unsqueeze(0).float()

        return image_A, image_B, mask


In [ ]:
# paths
image_dir_A = '/kaggle/input/preprocessed-change/change_detection_dataset_poly_preprocessed/newA'
image_dir_B = '/kaggle/input/preprocessed-change/change_detection_dataset_poly_preprocessed/newB'
mask_dir = '/kaggle/input/preprocessed-change/change_detection_dataset_poly_preprocessed/label'

transform = transforms.Compose([
    transforms.ToTensor()
])

# Creating dataset
dataset = ChangeDetectionDataset(image_dir_A, image_dir_B, mask_dir, transform=transform)

In [ ]:
def visualize_samples(dataset, num_samples=5):
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5 * num_samples))
    dataset_size = len(dataset)
    indices = np.random.choice(dataset_size, num_samples, replace=False)

    for i, idx in enumerate(indices):
        image_A, image_B, mask = dataset[idx]
        image_A = image_A.permute(1, 2, 0).numpy()
        image_B = image_B.permute(1, 2, 0).numpy()
        mask = mask.squeeze(0).numpy()

        axes[i, 0].imshow(image_A)
        axes[i, 0].set_title('Image A')
        axes[i, 0].axis('off')

        axes[i, 1].imshow(image_B)
        axes[i, 1].set_title('Image B')
        axes[i, 1].axis('off')

        axes[i, 2].imshow(mask, cmap='gray')
        axes[i, 2].set_title('Mask')
        axes[i, 2].axis('off')

    plt.tight_layout()
    plt.show()

visualize_samples(dataset)


In [ ]:
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()

        def conv_block(in_channels, out_channels):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True)
            )

        def upconv_block(in_channels, out_channels):
            return nn.Sequential(
                nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
                nn.ReLU(inplace=True)
            )

        self.encoder1 = conv_block(in_channels, 64)
        self.encoder2 = conv_block(64, 128)
        self.encoder3 = conv_block(128, 256)
        self.encoder4 = conv_block(256, 512)

        self.pool = nn.MaxPool2d(2)

        self.bottleneck = conv_block(512, 1024)

        self.upconv4 = upconv_block(1024, 512)
        self.decoder4 = conv_block(1024, 512)
        self.upconv3 = upconv_block(512, 256)
        self.decoder3 = conv_block(512, 256)
        self.upconv2 = upconv_block(256, 128)
        self.decoder2 = conv_block(256, 128)
        self.upconv1 = upconv_block(128, 64)
        self.decoder1 = conv_block(128, 64)

        self.conv_last = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x1, x2):
        x = torch.cat([x1, x2], dim=1)

        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool(enc1))
        enc3 = self.encoder3(self.pool(enc2))
        enc4 = self.encoder4(self.pool(enc3))

        bottleneck = self.bottleneck(self.pool(enc4))

        dec4 = self.upconv4(bottleneck)
        dec4 = self.decoder4(torch.cat((dec4, enc4), dim=1))
        dec3 = self.upconv3(dec4)
        dec3 = self.decoder3(torch.cat((dec3, enc3), dim=1))
        dec2 = self.upconv2(dec3)
        dec2 = self.decoder2(torch.cat((dec2, enc2), dim=1))
        dec1 = self.upconv1(dec2)
        dec1 = self.decoder1(torch.cat((dec1, enc1), dim=1))

        return self.conv_last(dec1)

In [ ]:
# Splitting the dataset into training, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Creating data loaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Initializing the model, loss function, and optimizer
model = UNet(in_channels=6, out_channels=1).cuda()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    running_train_loss = 0.0
    for images_A, images_B, masks in tqdm(train_loader):
        images_A = images_A.cuda()
        images_B = images_B.cuda()
        masks = masks.cuda()

        optimizer.zero_grad()
        outputs = model(images_A, images_B)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

    avg_train_loss = running_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Validation loop
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for images_A, images_B, masks in val_loader:
            images_A = images_A.cuda()
            images_B = images_B.cuda()
            masks = masks.cuda()

            outputs = model(images_A, images_B)
            loss = criterion(outputs, masks)
            running_val_loss += loss.item()

    avg_val_loss = running_val_loss / len(val_loader)
    val_losses.append(avg_val_loss)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}")

# Plotting the training and validation loss
plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss')
plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# Evaluating the model on the test set
model.eval()
test_loss = 0.0
inference_times = []
with torch.no_grad():
    for images_A, images_B, masks in tqdm(test_loader):
        images_A = images_A.cuda()
        images_B = images_B.cuda()
        masks = masks.cuda()

        start_time = time.time()
        outputs = model(images_A, images_B)
        end_time = time.time()

        inference_time = end_time - start_time
        inference_times.append(inference_time)

        loss = criterion(outputs, masks)
        test_loss += loss.item()

avg_test_loss = test_loss / len(test_loader)
avg_inference_time = np.mean(inference_times)
print(f"Test Loss: {avg_test_loss}")
print(f"Average Inference Time per Batch: {avg_inference_time:.6f} seconds")


In [ ]:
def predict_change_mask(model, image_A_path, image_B_path, transform, device="cuda"):
    # Loading and preprocessing images
    image_A = Image.open(image_A_path).convert('RGB')
    image_B = Image.open(image_B_path).convert('RGB')
    image_A = transform(image_A).unsqueeze(0).to(device)
    image_B = transform(image_B).unsqueeze(0).to(device)

    # predictions
    model.eval()
    with torch.no_grad():
        output = model(image_A, image_B)

    # Threshold and convertion to binary mask
    mask = (output > 0.05).float().cpu().numpy()[0, 0]
    return mask

In [ ]:
# Prediction Example
image_A_path = "/kaggle/input/preprocessed-change/change_detection_dataset_poly_preprocessed/newA/79120_50988.jpg"
image_B_path = "/kaggle/input/preprocessed-change/change_detection_dataset_poly_preprocessed/newB/79120_50988.jpg"

change_mask = predict_change_mask(model, image_A_path, image_B_path, transform)

plt.imshow(change_mask, cmap='gray')